In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('arquivos\dadosalunos-limpo.csv')

C:\Users\romul\AppData\Local\Temp\ipykernel_7980\1396285713.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('arquivos\dadosalunos-limpo.csv')


In [29]:
df.columns

Index(['ANO_OCORRENCIA', 'CD_ALUNO', 'CD_CURSO', 'CD_ORIGEM_ALUNO', 'CD_TURNO',
       'DE_NATURALIDADE', 'DH_MATRICULA', 'DH_SITUACAO', 'DT_ANO_INGRES',
       'DT_NASC_ALUNO', 'LT_ESTADO_CIVIL', 'LT_MODALIDADE', 'LT_NACIONALIDADE',
       'LT_RACA', 'NM_CIDADE_AULA', 'NM_CIDADE_EST2GRAU', 'NM_CIDADE_FERIAS',
       'NM_CIDADE_NASC', 'NM_PAIS_NASC', 'NU_SEMESTRE2_VEST', 'SG_SEXO_ALUNO',
       'SG_UF_AULA', 'SG_UF_EST2GRAU', 'SG_UF_FERIAS', 'SG_UF_NASC_ALUNO',
       'ST_ALUNO', 'TP_DEFIC_INEP', 'TP_ESCOLA', 'TP_INGRES_CURSO',
       'TP_OPCAO_COTAS', 'F_CD_ORIGEM_ALUNO', 'F_DT_NASC_ALUNO',
       'F_NM_PAIS_NASC', 'F_NU_SEMESTRE2_VEST', 'F_SG_UF_FERIAS',
       'F_SG_UF_NASC_ALUNO', 'BOOL_TRANCADO', 'BOOL_ST_ALUNO',
       'MULTI_ST_ALUNO', 'BOOL_DEFIC_INEP', 'F_TP_ESCOLA', 'F_TP_OPCAO_COTAS',
       'BOOL_COTAS', 'IDADE_MATRICULA', 'ANO_NASCIMENTO',
       'MESES_ATE_ULTIMA_SITUACAO', 'NM_CURSO', 'TP_HABILITACAO', 'SG_CENTRO',
       'NM_CAMPUS', 'MAX_ANOS', 'BOOL_SEDE', 'LOCAL_FERI

In [45]:
colunas_excluir_ml = ['NM_CAMPUS', 
                      'SG_CENTRO', 
                      'NM_CURSO',
                      'GERACAO_COMPLETA', 
                      'NUMERO_GERACAO', 
                      'BOOL_ST_ALUNO', 
                      'MULTI_ST_ALUNO']
colunas_ml = ['CD_CURSO',
         'CD_TURNO',
         'LT_ESTADO_CIVIL',
         'LT_RACA',
         'F_NU_SEMESTRE2_VEST',
         'SG_SEXO_ALUNO',
         'BOOL_DEFIC_INEP',
         'F_TP_ESCOLA',
         'BOOL_COTAS',
         'IDADE_MATRICULA',
         'LOCAL_FERIAS',
         'LOCAL_NASC']
target_bool = 'BOOL_ST_ALUNO'
target_multi = 'MULTI_ST_ALUNO'

In [46]:
df = df[colunas_ml + colunas_excluir_ml].copy()

In [32]:
df = df.dropna()
df.shape[0]

66820

### Preparação do Dataset

In [48]:
pd.cut(df['IDADE_MATRICULA'], bins=[0, 16, 18, 20, 22, 24, 26, 30, 40, 50, 60, 70]).value_counts()

IDADE_MATRICULA
(16, 18]    36249
(18, 20]    14251
(20, 22]     5671
(22, 24]     3226
(26, 30]     2130
(24, 26]     1879
(30, 40]     1786
(0, 16]       935
(40, 50]      554
(50, 60]      128
(60, 70]       10
Name: count, dtype: int64

In [49]:
df['IDADE_MATRICULA'] = pd.cut(df['IDADE_MATRICULA'], bins=[0, 16, 18, 20, 22, 24, 26, 30, 40, 50, 60, 70]).value_counts()

In [33]:
from sklearn.preprocessing import LabelEncoder

In [51]:
for coluna in colunas_ml:
    le = LabelEncoder()
    df[coluna] = le.fit_transform(df[coluna])

### Modelos

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from sklearn.metrics import classification_report, accuracy_score, f1_score

#### Árvore de Decisão e Floresta Aleatória

In [53]:
X = df.drop(colunas_excluir_ml, axis=1)
y = df[target_bool]

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [58]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [59]:
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

In [60]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10]
}

param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=cv, n_jobs=-1, verbose=2)

In [ ]:
grid_search_dt.fit(X_train, y_train)

In [ ]:
y_pred_dt = grid_search_dt.best_estimator_.predict(X_test)

In [ ]:
print("Melhores parâmetros para Árvore de Decisão:", grid_search_dt.best_params_)
print("Relatório de Classificação para Árvore de Decisão:\n", classification_report(y_test, y_pred_dt))